In [1]:
# Install the environnement
%pip install git+https: // github.com/AwePhD/NotebooksLabsessionImage.git


  Cloning https://github.com/AwePhD/NotebooksLabsessionImage.git to /tmp/pip-req-build-wko581wo
  Running command git clone -q https://github.com/AwePhD/NotebooksLabsessionImage.git /tmp/pip-req-build-wko581wo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for NLI: filename=NLI-1.0.0-py3-none-any.whl size=2406 sha256=74aa5692ff28a7ace01deb5f6b2999d40864cd7ab8900ccf3958945ce986ccb2
  Stored in directory: /tmp/pip-ephem-wheel-cache-7y8z23vr/wheels/17/4a/a4/4f920391e876c3c2632ecc7851748e1c11539349fe2eefd2c4
Successfully built NLI


In [2]:
# Import dataset
# Can be found at https://www.kaggle.com/vishalsubbiah/pokemon-images-and-types
!rm - rf ./*
!curl - LO https: // github.com/AwePhD/NotebooksLabsessionImage/raw/main/pokemon_dataset.zip
!unzip - qq pokemon_dataset.zip
!rm pokemon_dataset.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   156  100   156    0     0   1695      0 --:--:-- --:--:-- --:--:--  1695
100 2484k  100 2484k    0     0  10.6M      0 --:--:-- --:--:-- --:--:-- 10.6M


In [1]:
# Standard import
from pathlib import Path
# Third party imports
from skimage import io
import numpy as np
import pandas as pd

## Computation on `DataFrame` objects

`pandas` uses `numpy` under the hood for processing data. Computation on `pandas` are - if well used - vectorised and so optimal. So let's see what we can do. 🤓

### Create numeric data

For each Pokemon we will compute the mean color of the image.

In [2]:
pokemons = pd.read_csv("./pokemon.csv", index_col='Name')

pokemons_mean_colors = dict()
for filepath in Path('./images/').iterdir():
    pokemon_img = io.imread(filepath)
    if filepath.suffix == '.png':  # If they have alpha channel then we ignore the pixel which are transparent
        pokemons_mean_colors[filepath.stem] = pokemon_img[pokemon_img[:, :, 3] != 0].mean((0, 1))
    else:  # We consider pure white pixels as background
        pokemons_mean_colors[filepath.stem] = np.array([
            pixel
            for pixel in pokemon_img.reshape(-1, 3) # Reshape for dimensios compatiblity
            if (pixel != [255, 255, 255]).all() # Exlude pure white pixels
        ]).mean(0)


In [23]:
colour_names = ["Red", "Blue", "Green"]
for pokemon_name, mean_colour in pokemons_mean_colors.items():
    pokemons.loc[pokemon_name, colour_names] = mean_colour
    
pokemons.head()

,Type1,Type2,Red,Blue,Green
Name,,,,,
bulbasaur,Grass,Poison,184.842114,184.842114,184.842114
ivysaur,Grass,Poison,176.105307,176.105307,176.105307
venusaur,Grass,Poison,183.148531,183.148531,183.148531
charmander,Fire,NaN,188.954315,188.954315,188.954315
charmeleon,Fire,NaN,182.036871,182.036871,182.036871


Oof 😮‍💨

### Get stats

Back to method `.describe`


In [24]:
pokemons.describe()

,Red,Blue,Green
count,809.000000,809.000000,809.000000
mean,175.333047,174.720034,174.106963
std,21.330915,21.676132,22.372165
min,92.704551,95.195009,87.845656
25%,161.332639,161.293049,160.941354
50%,175.965098,175.841014,175.137358
75%,189.660389,189.095003,188.668950
max,229.714072,229.714072,229.714072


We can normalize them for example with `.std` and `.mean`

In [26]:
pokemons[colour_names] = (pokemons[colour_names] - pokemons[colour_names].mean()) / pokemons[colour_names].std()
pokemons.describe()

,Red,Blue,Green
count,8.090000e+02,8.090000e+02,809.000000
mean,1.756595e-17,1.756595e-17,0.000000
std,1.000000e+00,1.000000e+00,1.000000
min,-3.873650e+00,-3.668783e+00,-3.855742
25%,-6.563435e-01,-6.194364e-01,-0.588482
50%,2.963074e-02,5.171496e-02,0.046057
75%,6.716703e-01,6.631704e-01,0.650898
max,2.549399e+00,2.537078e+00,2.485549


Also we can do some *useful* :eyes: computation easily ...


In [49]:
query_color = pokemons.loc['pikachu'][colour_names]

# Top 5 index names which match pikachu colour
(query_color - pokemons[colour_names]).pow(2).sum('columns').sort_values().head(5).index

Index(['pikachu', 'mankey', 'omastar', 'mienshao', 'mareep'], dtype='object', name='Name')

In this computation there are several commonly used methods. They can be chained because each time they return a `Series` object which can call other methods.

Using `pandas` can make code readable, maintanable and efficient. 🥧